In [10]:
import os
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense, Flatten
import tensorflow.python.keras.optimizers
import tensorflow.python.keras.metrics

import keras_tuner as kt

#%%capture
data_file_path = os.path.join(os.getcwd(), "Data\\returnsData.csv")
returnsData = pd.read_csv(data_file_path)
#returnsData.info(verbose=True)
longReturns = returnsData[["YearQuarter", "gvkey", "quarterlyReturns"]] #"quarterlyVolatility",
#longReturns.info(verbose=True)
wideReturns = pd.pivot(longReturns, index="YearQuarter", columns="gvkey", values="quarterlyReturns")
#wideReturns.info(verbose=True)
#print(wideReturns.iloc[0:10,0:10])

#wideReturns["YearQuarter"] = wideReturns.index
#wideReturns["YQ"] = wideReturns.index

trainSize = math.floor(len(wideReturns.index) * 0.5)
validationSize = math.floor(len(wideReturns.index) * 0.75)
testSize = len(wideReturns.index)
# trainSize = len(wideReturns["YearQuarter"]) * 0.5
# validationSize = len(wideReturns["YearQuarter"]) * 0.75
#wideReturns[:,"Split"] = 

# normlayer = tf.keras.layers.Normalization(axis=None)
# #normlayer.adapt(xTrain[:,0,:])
# normlayer.adapt(wideReturns[trainSize, :])


sample_length = 10
dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=wideReturns, targets=None,
    sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
    batch_size=128,  shuffle=False, 
    start_index=None, end_index=None)

for batch in dataset:
  data = batch
  #assert np.array_equal(inputs[0], wideReturns[:sample_length])
  # second sample equals output timestamps 20-40
  #assert np.array_equal(targets[1], wideReturns[sample_length:2*sample_length])
  break

print(data.shape)
# print(dataset.cardinality())
# print(dataset.cache())

#data = data[~np.isnan(data).any(axis=2),:]
trainSize = math.floor(data.shape[0] * 0.5)
validationSize = math.floor(data.shape[0] * 0.75)
testSize = data.shape[0]

# xTrain = data[:(trainSize-sample_length), :-1, :]
# yTrain = data[sample_length:trainSize, -1, :]

# xVal = data[(trainSize-sample_length):(validationSize-sample_length), :-1, :]
# yVal = data[trainSize:validationSize, -1, :]

# xTest = data[validationSize-sample_length:-sample_length, :-1, :]
# yTest = data[validationSize:, -1, :]

# xTrain = data[:trainSize, :-1, :]
# yTrain = data[:trainSize, -1, :]

# xVal = data[trainSize:validationSize, :-1, :]
# yVal = data[trainSize:validationSize, -1, :]

# xTest = data[validationSize:, :-1, :]
# yTest = data[validationSize:, -1, :]

# yTrain = np.expand_dims(yTrain, axis=1)
# yVal = np.expand_dims(yVal, axis=1)
# yTest = np.expand_dims(yTest, axis=1)

train = data[:trainSize, :, :]
validation = data[trainSize:validationSize, :, :]
test = data[validationSize:, :, :]

def preprocessData(rawData):
  #[lags of YearQuarter, YearQuarter, gvkey] => [Batch, YearQuarter, None]
  reshapedData = np.transpose(a=rawData, axes=(2, 0, 1))
  reshapedData = np.reshape(reshapedData, (-1, rawData.shape[1]))
  reshapedData = reshapedData[~np.isnan(reshapedData).any(axis=1),:]
  reshapedData = np.expand_dims(reshapedData, axis=2)
  return reshapedData

# xTrainReshaped = preprocessData(xTrain)
# yTrainReshaped = preprocessData(yTrain)
# xValReshaped = preprocessData(xVal)
# yValReshaped = preprocessData(yVal)
# xTestReshaped = preprocessData(xTest)
# yTestReshaped = preprocessData(yTest)

train = preprocessData(train)
validation = preprocessData(validation)
test = preprocessData(test)

xTrain, yTrain = train[:, :-1, :], train[:, -1, :]
xVal, yVal = validation[:, :-1, :], validation[:, -1, :]
xTest, yTest = test[:, :-1, :], test[:, -1, :]

# train = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xTrainReshaped, targets=yTrainReshaped,
#     sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)
# validation = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xValReshaped, targets=yValReshaped,
#     sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)
# test = tf.keras.preprocessing.timeseries_dataset_from_array(
#     data=xTestReshaped, targets=yTestReshaped,
#     sequence_length=sample_length, sequence_stride=1, sampling_rate=1,
#     batch_size=batch_size,  shuffle=False)

#scaledData = normlayer(reshapedData)

# train = preprocessData(None, trainSize)
# val = preprocessData(trainSize, validationSize)
# test = preprocessData(validationSize, None)

# xTrain, xVal, xTest = train[:,:-1,:], val[:,:-1,:], test[:,:-1,:]
# yTrain, yVal, yTest = train[:,-1,:], val[:,-1,:], test[:,-1,:]





(44, 10, 9668)


In [12]:
model = Sequential()
#model.add(normlayer)
model.add(tf.keras.layers.LSTM(units=5)) #, input_shape=(,sample_length-1)   #mask
model.add(Dense(units=1))

model.compile(optimizer="adam", #tf.keras.optimizers.Adam(learning_rate=0.001)
              loss="MeanAbsoluteError",
              metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])

#model.fit(x=xTrain, y=yTrain, batch_size=32, validation_data=(xVal, yVal))
#model.fit(x=dataset, batch_size=8)#, validation_data=(xVal, yVal))
model.fit(x=xTrain, y=yTrain, batch_size=32, validation_data=(xVal, yVal))

# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(tf.keras.layers.LSTM(units=hp.Int("units", min_value=32, max_value=512, step=32), input_shape=(sample_length-1,)))
#         model.add(Dense(units=1))

#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Int("learning_rate", [16, 32])),
#                   loss="MeanAbsoluteError",
#                   metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError()])
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             batch_size=hp.Choice("batch_size", [16, 32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )

 460/4465 [==>...........................] - ETA: 29s - loss: 15.4126 - mean_squared_error: 707.0060 - mean_absolute_error: 15.4126

KeyboardInterrupt: 